### Convert the hourly ERA5 fields to daily and save as netCDF files

A bit of data processing... Now we read in each of the 1-hour ERA5 data files and create daily means. This is because for the storm statistics, we use the daily mean variables so we don't care too much about sub-daily variability. This also makes the data much, much easier to work with. We also cut the latitudes at 40S because we don't use data north of that due to the strong subtropical, and western boundary current infleunce.

In [1]:
# load the modules needed for the data processing

import sys
sys.path.append('../functions/')

import numpy as np
import xarray as xr

from glob import glob
from tqdm.notebook import tqdm 

from storm_interstorm_id import storm_interstorm_id

from adjust_lon_xr_dataset import *

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [13]:
file_list

['/Volumes/LaCie/Work/data/era5/DJF_1981_2023_hourly_means/DJF_1981_2023_net_solar_radiation/surface_net_solar_radiation_198101.nc',
 '/Volumes/LaCie/Work/data/era5/DJF_1981_2023_hourly_means/DJF_1981_2023_net_solar_radiation/surface_net_solar_radiation_198102.nc',
 '/Volumes/LaCie/Work/data/era5/DJF_1981_2023_hourly_means/DJF_1981_2023_net_solar_radiation/surface_net_solar_radiation_198112.nc',
 '/Volumes/LaCie/Work/data/era5/DJF_1981_2023_hourly_means/DJF_1981_2023_net_solar_radiation/surface_net_solar_radiation_198201.nc',
 '/Volumes/LaCie/Work/data/era5/DJF_1981_2023_hourly_means/DJF_1981_2023_net_solar_radiation/surface_net_solar_radiation_198202.nc',
 '/Volumes/LaCie/Work/data/era5/DJF_1981_2023_hourly_means/DJF_1981_2023_net_solar_radiation/surface_net_solar_radiation_198212.nc',
 '/Volumes/LaCie/Work/data/era5/DJF_1981_2023_hourly_means/DJF_1981_2023_net_solar_radiation/surface_net_solar_radiation_198301.nc',
 '/Volumes/LaCie/Work/data/era5/DJF_1981_2023_hourly_means/DJF_1981_2

In [14]:
data_directory_out = '/Volumes/LaCie/Work/data/era5/DJF_1981_2023_daily_means/'

vars = [
    # 'winds', 
    # 'latent_heat_flux', 
    # 'sensible_heat_flux', 
    # '2m_dewpoint_temperature', 
    # '2m_temperature', 
    'net_solar_radiation',
    'net_thermal_radiation',
    # 'sea_ice_cover', 
    # 'sea_surface_temperature'
]

for var in vars:

    data_directory_in  = '/Volumes/LaCie/Work/data/era5/DJF_1981_2023_hourly_means/DJF_1981_2023_' + var + '/*.nc'

    file_list = sorted(glob(data_directory_in))

    print(str(var))
    
    years = np.arange(2022, 2023)
    
    for y in tqdm(range(years.size)):
    
        year = years[y]
        print(year)
    
        D = str(year) + '12.nc'
        J = str(year+1) + '01.nc'
        F = str(year+1) + '02.nc'
        
        # choose only the files that are dec, jan or feb
        filtered_files = [file for file in file_list if file.endswith(D) or file.endswith(J) or file.endswith(F)]
        
        # load them into xarray
        ds = xr.open_mfdataset(filtered_files, engine='netcdf4')
    
        # load the dataset
        ds = ds.load()    

        # cut the latitudes to our definition of the southern ocean
        ds = ds.sel(latitude=slice(-40, -90))

        # convert the fluxes to W m-2
        if var == 'net_solar_radiation':
        
            ds['ssr'] = (('time', 'latitude', 'longitude'), (ds['ssr'] / 3600).data) 

        if var == 'net_thermal_radiation':
        
            ds['str'] = (('time', 'latitude', 'longitude'), (ds['str'] / 3600).data)                 
    
        # as this stage - resample the data to daily resolution, then do the calculations and adjust longitudes

        ds_1D = ds.resample(time='1D').mean()
        
        if var == 'winds':
            
            # calculate the wind speed
            ds_1D['ws'] = (('time', 'latitude', 'longitude'), np.sqrt(ds_1D['u10'].data**2 + ds_1D['v10'].data**2))
        
        # Adjust the longitudes to be -180 to 180
        ds_1D = adjust_lon_xr_dataset(ds_1D)    
    
        print('saving to file...')
        
        ds_1D.to_netcdf(data_directory_out + str(var) + '_' + str(year) + '_DJF.nc')

net_solar_radiation


  0%|          | 0/1 [00:00<?, ?it/s]

2022
saving to file...
net_thermal_radiation


  0%|          | 0/1 [00:00<?, ?it/s]

2022
saving to file...
